# filter()
- `DataFrameGroupBy.filter(func, dropna=True, *args, **kwargs)`
- 특정 집계 조건을 만족하는 Group의 행들만 조회한다.
    1. DataFrameGroupBy의 group로 DataFrame을 함수에 전달한다.
    2. 함수는 받은 DataFrame을 이용해 집계한 값의 조건을 비교해서 반환한다.(반환타입: Bool) 
    3. 반환값이 True인 Group들의 모든 행들로 구성된 DataFrame을 반환한다.
- 매개변수
    - func: filtering 조건을 구현한 함수
        - 첫번째 매개변수로 Group으로 묶인 DataFrame을 받는다.
    - dropna=True
        - 필터를 통과하지 못한 group의 DataFrame의 값들을 drop시킨다. False로 설정하면 NA 처리해서 반환한다.
    - \*args, \*\*kwargs: filter 함수의 매개변수에 전달할 전달인자값.

In [1]:
import numpy as np
import pandas as pd

In [3]:
# cnt1 - 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


#### 과일중 cnt1의 평균이 20 이상인 과일들만 보기
- 집계결과가 아니라 조건을 만족하는 행들을 다 출력

In [7]:
def check_cnt1_mean(x):
    """
    filter에 사용할 함수
    매개변수: x - DataFrame(group별로 나뉜)
    반환값: bool - 특정 조건을 만족하는지 여부
    """
    return x['cnt1'].mean() >= 20
# group별로 특정 조건을 만족하는 행들을 조회 해주는 함수
df.groupby('fruits').filter(check_cnt1_mean)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [8]:
# Lambda 식으로 구현
df.groupby('fruits').filter(lambda x : x['cnt1'].mean()>=20)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


#### 매개변수 있는 filter함수

In [9]:
def check_mean(x,col_name,mean_thresh):
    """
    [Parameter]
        - x: DataFrame - group별 DataFrame
        - col_name: str - 평균을 계산할 컬럼명
        - mean_thresh: int - 컬럼의 평균값이 이 값 이상인지 비교할 기준값
    [return]
        - bol: 컬럼의 평군이 mean_thresh 이상인지 여부
    """
    return x[col_name].mean() >= mean_thresh

In [10]:
df.groupby('fruits').filter(check_mean,col_name='cnt2',mean_thresh=100)

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


# transform
함수에 의해 처리된 값(반환값)으로 원래 값들을 변경(tranform) 해서 반환    
DataFrame에 Group 단위 통계량을 추가할 때 유용하다.
- `DataFrameGroupBy.transform(func, *args)`, `SeriesGroupBy.transform(func, *args)`
    - func: 매개변수로 그룹별로 Series를 받아 Series의 값들을 변환하여 (Series로)반환하는 함수객체
        - DataFrameGroupBy은 모든 컬럼의 값들을 group 별 Series로 전달한다.
    - *args: 함수에 전달할 추가 인자값이 있으면 매개변수 순서에 맞게 값을 전달한다. (위치기반 argument)
- transform() 함수를 groupby() 와 사용하면 컬럼의 각 원소들을 자신이 속한 그룹의 통계량으로 변환된 데이터셋을 생성할 수 있다.
- 컬럼의 값과 통계값을 비교해서 보거나 결측치 처리등에 사용할 수있다.

In [11]:
# cnt1 평균
df.groupby('fruits')['cnt1'].mean()

fruits
귤     24.0
딸기    34.2
배      5.4
사과    11.6
Name: cnt1, dtype: float64

In [12]:
# agg(), filter(), transform - DataFrameGroupby의; 메소드: 판다스 제공 집계함수 -> 문자열
#                                                          사용자정의 집계함수 -> 함수객체
df.groupby('fruits').transform('mean')

,cnt1,cnt2
0,11.6,103.6
1,11.6,103.6
2,11.6,103.6
3,11.6,103.6
4,11.6,103.6
5,24.0,54.8
6,24.0,54.8
7,24.0,54.8
8,24.0,54.8
9,24.0,54.8


In [13]:
# lambda식
df.groupby('fruits').transform(lambda x : x.max()-x.min())

,cnt1,cnt2
0,3,7
1,3,7
2,3,7
3,3,7
4,3,7
5,6,7
6,6,7
7,6,7
8,6,7
9,6,7


## 원본에 통계치 붙여서 비교하기

In [14]:
 # cnt1의 과일별 평균을 df에 행별로 붙이기
cnt1_mean = df.groupby('fruits')['cnt1'].transform('mean')
cnt1_mean

0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

In [15]:
# 컬럼을 삽입할 위치, 컬럼명, 컬럼값 ->  원본을 변경
df.insert(2, 'cnt mean',cnt1_mean)
df

,fruits,cnt1,cnt mean,cnt2
0,사과,10,11.6,100
1,사과,12,11.6,103
2,사과,13,11.6,107
3,사과,11,11.6,107
4,사과,12,11.6,101
5,귤,21,24.0,51
6,귤,22,24.0,57
7,귤,27,24.0,58
8,귤,24,24.0,57
9,귤,26,24.0,51


## 결측치 처리
- 결측치: 모르는 값, 없는 값
- 결측치 처리
    - 제거(행/열단위)
        - 데이터양이 충분할때 제거
    - 대체
        - 가장 가능성이 높은 값을 대체)평균, 중앙값, 최빈값)
        - 결측치를 표현하는 값으로 대체(없다라는 값으로 변경)
- transform이용해서 여기선 결측치를 같은 과일그룹의 평균값으로 변환
    - 전체 평균보다 좀더 정확할 수 있다.

In [16]:
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)

df.loc[[0,1,5,7,10,12,15,16],'cnt2'] = None

df

,fruits,cnt1,cnt2
0,사과,10,NaN
1,사과,12,NaN
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,NaN
6,귤,22,57.0
7,귤,27,NaN
8,귤,24,57.0
9,귤,26,51.0


#### 결측치제거 - dropna()
- df.dropna(): 결측치가 있는 행(기본) 제거
- df.dropna(axis=1): 결측치가 있는 열을 제거 

In [17]:
df.dropna()  # 결측치가 있는 행(기본) 제거

,fruits,cnt1,cnt2
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
6,귤,22,57.0
8,귤,24,57.0
9,귤,26,51.0
11,배,7,9.0
13,배,3,7.0
14,배,2,7.0
17,딸기,37,213.0


#### fillna(대체값)

In [19]:
df2 = df.copy()
df2['cnt2'] = df['cnt2'].fillna(round(df['cnt2'].mean(),2))  # Nan값을 cnt2의 평균값으로 대체
df2

,fruits,cnt1,cnt2
0,사과,10,93.83
1,사과,12,93.83
2,사과,13,107.00
3,사과,11,107.00
4,사과,12,101.00
5,귤,21,93.83
6,귤,22,57.00
7,귤,27,93.83
8,귤,24,57.00
9,귤,26,51.00


#### transform 사용

In [20]:
df['cnt2'].fillna(df.groupby('fruits')['cnt2'].transform('mean'), inplace=True)
df  # 과일별 cnt2의 평균값을 Na값에 넣어준다

,fruits,cnt1,cnt2
0,사과,10,105.000000
1,사과,12,105.000000
2,사과,13,107.000000
3,사과,11,107.000000
4,사과,12,101.000000
5,귤,21,55.000000
6,귤,22,57.000000
7,귤,27,55.000000
8,귤,24,57.000000
9,귤,26,51.000000


# pivot_table()
엑셀의 pivot table 기능을 제공하는 메소드.    
분류별 집계(Group으로 묶어 집계)를 처리하는 함수로 group으로 묶고자 하는 컬럼을 행과 열로 위치시키고 집계값을 값으로 보여준다.    
역할은 groupby() 를 이용한 집계와 같다.

> pivot() 함수와 역할이 다르다.   
> pivot() 은 index와 column의 형태를 바꾸는 reshape 함수.

- `DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`
- **매개변수**
    - index
        - 문자열 또는 리스트. index로 올 컬럼들 => groupby였으면 묶었을 컬럼
    - columns
        - 문자열 또는 리스트. column으로 올 컬럼들 => groupby였으면 묶었을 컬럼 (index/columns가 묶여서 groupby에 묶을 컬럼들이 된다.)
    - values
        - 문자열 또는 리스트. 집계할 대상 컬럼들
    - aggfunc
        - 집계함수 지정. 함수, 함수이름문자열, 함수리스트(함수이름 문자열/함수객체), dict: 집계할 함수
        - 기본(생략시): 평균을 구한다. (mean이 기본값)
    - fill_value, dropna
        - fill_value: 집계시 NA가 나올경우 채울 값
        - dropna: boolean. 컬럼의 전체값이 NA인 경우 그 컬럼 제거(기본: True)
    - margins/margins_name
        - margin: boolean(기본: False). 총집계결과를 만들지 여부.
        - margin_name: margin의 이름 문자열로 지정 (생략시 All)

In [26]:
flights = pd.read_csv('data/flights.csv')
flights

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58487,12,31,4,AA,SFO,DFW,515,5.0,166.0,1464,1045,-19.0,0,0
58488,12,31,4,F9,LAS,SFO,1910,13.0,71.0,414,2050,4.0,0,0
58489,12,31,4,OO,SFO,SBA,1846,-6.0,46.0,262,1956,-5.0,0,0
58490,12,31,4,WN,MSP,ATL,525,39.0,124.0,907,855,34.0,0,0


## 1개의 컬럼을 grouping 해서 집계
- 항공사별 비행시간의 평균 
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
    - 집계대상컬럼
        - AIR_TIME
- 집계: mean

In [22]:
# groupby
flights.groupby('AIRLINE')['AIR_TIME'].mean()

AIRLINE
AA    144.259404
AS    147.845052
B6    209.412963
DL    115.334187
EV     68.964016
F9    127.592337
HA    338.288288
MQ     61.318346
NK    135.736878
OO     76.010272
UA    155.650521
US    147.686755
VX    154.864097
WN    107.005897
Name: AIR_TIME, dtype: float64

In [23]:
flights.pivot_table(values='AIR_TIME',   #집계대상
                    index='AIRLINE',   # GROUPING할 대상 컬럼 중 INDEX로 올 컬럼 
#                     columns='AIRLINE',
                    aggfunc='mean' ,    #집계 함수 - default: mean
                    margins=True, # 총집계
                    margins_name='전체 평균'
                   )

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


## 두개의 컬럼을 grouping 해서 집계
- 항공사/출발공항코드 별 취소 총수 (1이 취소이므로 합계를 구한다.)
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
        - ORG_AIR: 출발 공항코드
    - 집계대상컬럼
        - CANCELLED: 취소여부 - 1:취소, 0: 취소안됨
- 집계: sum

In [24]:
flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [25]:
flights.pivot_table(values='CANCELLED', 
                    index='AIRLINE', columns='ORG_AIR', # 그룹으로 묶을 컬럼들을 INDEX, COLOUMNS지정
                    aggfunc='sum', 
                    margins=True
#                     fill_value=-10000  # NaN을 지정한 값으로 변경
                    
                   )
# NaN인 것은 그 group들의 값을 가진 행이 없는 경우.

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO,All
AIRLINE,,,,,,,,,,,
AA,3.0,4.0,86.0,3.0,3.0,11.0,3.0,35.0,4.0,2.0,154
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
B6,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,1
DL,28.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0,38
EV,18.0,6.0,27.0,36.0,NaN,NaN,6.0,53.0,0.0,NaN,146
F9,0.0,2.0,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0,10
HA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0
MQ,5.0,NaN,62.0,0.0,NaN,0.0,0.0,85.0,NaN,NaN,152
NK,1.0,1.0,6.0,0.0,1.0,1.0,3.0,10.0,2.0,NaN,25


## 3개 이상의 컬럼을 grouping해서 집계
- 항공사/월/출발공항코드 별 취소 총수 
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - CANCELLED: 취소여부
- 집계 : sum    

In [27]:
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR'])['CANCELLED'].sum()#[:20]

AIRLINE  MONTH  ORG_AIR
AA       1      ATL        0
                DEN        0
                DFW        8
                IAH        0
                LAS        0
                          ..
WN       12     LAS        1
                LAX        2
                MSP        0
                PHX        0
                SFO        0
Name: CANCELLED, Length: 1133, dtype: int64

In [30]:
result = flights.pivot_table(values='CANCELLED', 
                            index=['AIRLINE', 'ORG_AIR'], 
                            columns='MONTH', 
                            aggfunc='sum')
result

MONTH             1     2     3    4    5    6    7    8    9    11   12
AIRLINE ORG_AIR                                                         
AA      ATL      0.0   2.0   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
        DEN      0.0   1.0   0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  1.0
        DFW      8.0  33.0  13.0  4.0  8.0  7.0  1.0  2.0  1.0  3.0  6.0
        IAH      0.0   1.0   0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
        LAS      0.0   2.0   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
...              ...   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...
WN      LAS      1.0   1.0   1.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
        LAX      3.0   2.0   3.0  2.0  1.0  0.0  9.0  4.0  3.0  3.0  2.0
        MSP      1.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
        PHX      0.0   2.0   1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
        SFO      4.0   5.0   0.0  2.0  2.0  6.0  0.0  2.0  2.0  2.0  0.0

[114 rows x 11 columns]

# apply() - Series, DataFrame의 데이터 일괄 처리

데이터프레임의 행들과 열들 또는 Series의 원소들에 공통된 처리를 할 때 apply 함수를 이용하면 반복문을 사용하지 않고 일괄 처리가 가능하다.

- DataFrame.apply(함수, axis=0, args=())
    - 인수로 행이나 열을 받는 함수를 apply 메서드의 인수로 넣으면 데이터프레임의 행이나 열들을 하나씩 함수에 전달한다.
    - 매개변수
        - 함수: DataFrame의 행들 또는 열들을 전달할 함수
        - axis: **0-열을 전달, 1-행을 전달 (기본값 0)** G: 0이 열이다...
        - args: 행/열 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
- Series.apply(함수, args=())
    - 인수로 Series의 원소들을 받는 함수를 apply 메소드의 인수로 넣으면  Series의 원소들을 하나씩 함수로 전달한다.
    - 매개변수
        - 함수: Series의 원소들을 전달할 함수
        - args: 원소 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달

In [31]:
import numpy as np
import pandas as pd

arr = np.arange(24).reshape(6,4)
df = pd.DataFrame(arr, columns=['no1', 'no2', 'no3', 'no4'])
df

,no1,no2,no3,no4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [32]:
# apply에 전달할 함수
def func(X):
    """
    매개변수 X 
    - dataframe.apply() -> Series 가 전달됨. (DF의 한행 또는 한열)
    - series.apply() -> series 원소가 전달.
    """
    print(X)
    print('---------------------------')
    return X**2

In [33]:
df.apply(func)

0     0
1     4
2     8
3    12
4    16
5    20
Name: no1, dtype: int32
---------------------------
0     1
1     5
2     9
3    13
4    17
5    21
Name: no2, dtype: int32
---------------------------
0     2
1     6
2    10
3    14
4    18
5    22
Name: no3, dtype: int32
---------------------------
0     3
1     7
2    11
3    15
4    19
5    23
Name: no4, dtype: int32
---------------------------


,no1,no2,no3,no4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


In [34]:
df.apply(func, axis=1) #axis=1: 행단위로 전달

no1    0
no2    1
no3    2
no4    3
Name: 0, dtype: int32
---------------------------
no1    4
no2    5
no3    6
no4    7
Name: 1, dtype: int32
---------------------------
no1     8
no2     9
no3    10
no4    11
Name: 2, dtype: int32
---------------------------
no1    12
no2    13
no3    14
no4    15
Name: 3, dtype: int32
---------------------------
no1    16
no2    17
no3    18
no4    19
Name: 4, dtype: int32
---------------------------
no1    20
no2    21
no3    22
no4    23
Name: 5, dtype: int32
---------------------------


,no1,no2,no3,no4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


#### 추가 매개변수가 있는 경우

In [35]:
def func2(X, value):
    return X + value

In [36]:
df.apply(func2, value=100)

,no1,no2,no3,no4
0,100,101,102,103
1,104,105,106,107
2,108,109,110,111
3,112,113,114,115
4,116,117,118,119
5,120,121,122,123


In [37]:
df.apply(lambda X: X * 2)

,no1,no2,no3,no4
0,0,2,4,6
1,8,10,12,14
2,16,18,20,22
3,24,26,28,30
4,32,34,36,38
5,40,42,44,46


# cut()/qcut() - 연속형(실수)을 범주형으로 변환
- cut() : 지정한 값을 기준으로 구간을 나눠 그룹으로 묶는다.
    - `pd.cut(x, bins,right=True, labels=None)`
    - 매개변수
        - x: 범주형으로 바꿀 대상. 1차원 배열형태(Series, 리스트, ndarray)의 자료구조
        - bins: 범주로 나눌때의 기준값(구간경계)들을 리스트로 묶어서 전달한다.
        - right: 구간경계의 오른쪽(True-기본)을 포함할지 왼쪽(False)을 포함할지
        - labels: 각 구간(범주)의 label을 리스트로 전달
            - 생략하면 범위를 범주명으로 사용한다. (ex: (10, 20], ()-포함안함, []-포함)
- qcut() :  대상배열의 최대값 ~ 최소값을 지정한 개수의 동등한 size(**원소의개수**)가 되도록 나눈다.
    - `pd.qcut(x, q, labels)`
    - 매개변수
        - x: 나눌 대상. 1차원 배열형태의 자료구조
        - q: 나눌 개수
        - labels: 각 구간(범주)의 label을 리스트로 전달

In [38]:
ages = pd.Series(np.random.randint(50, size=30))  # 0 ~ 50 숫자중에 30개의 정수를 random하게 생성
ages

0     16
1      4
2     40
3      8
4     31
5     16
6     13
7      6
8     29
9     31
10     2
11    14
12     0
13    36
14     1
15    16
16    43
17    15
18    36
19    21
20    34
21     7
22    39
23    47
24     9
25    18
26    21
27    39
28    26
29    14
dtype: int32

#### cut

In [39]:
# cut()을 이용해 구간별로 그룹화
bins = [-1, 10, 30, 40, 50]
# -1~10, 10~30, 30~40, 40~50
# right:False - 작은쪽을 포함, True(기본): 큰쪽을 포함
pd.cut(ages, bins=bins, right=False)  #ages의 각 index의 값이 어느 그룹에 포함되는지를 반환

# (): 포함안함, []: 포함
# (-1, 10]   -1은 포함안함, 10은 포함

0     [10, 30)
1     [-1, 10)
2     [40, 50)
3     [-1, 10)
4     [30, 40)
5     [10, 30)
6     [10, 30)
7     [-1, 10)
8     [10, 30)
9     [30, 40)
10    [-1, 10)
11    [10, 30)
12    [-1, 10)
13    [30, 40)
14    [-1, 10)
15    [10, 30)
16    [40, 50)
17    [10, 30)
18    [30, 40)
19    [10, 30)
20    [30, 40)
21    [-1, 10)
22    [30, 40)
23    [40, 50)
24    [-1, 10)
25    [10, 30)
26    [10, 30)
27    [30, 40)
28    [10, 30)
29    [10, 30)
dtype: category
Categories (4, interval[int64, left]): [[-1, 10) < [10, 30) < [30, 40) < [40, 50)]

In [41]:
labels = ['10세이하', '10,20대', '30대', '40대']
# -1~10, 10~30, 30~40, 40~50
age_cate = pd.cut(ages, bins=bins,right=False, labels=labels)
age_cate

0     10,20대
1      10세이하
2        40대
3      10세이하
4        30대
5     10,20대
6     10,20대
7      10세이하
8     10,20대
9        30대
10     10세이하
11    10,20대
12     10세이하
13       30대
14     10세이하
15    10,20대
16       40대
17    10,20대
18       30대
19    10,20대
20       30대
21     10세이하
22       30대
23       40대
24     10세이하
25    10,20대
26    10,20대
27       30대
28    10,20대
29    10,20대
dtype: category
Categories (4, object): ['10세이하' < '10,20대' < '30대' < '40대']

In [42]:
age_df = pd.DataFrame({'나이':ages, '나이대':age_cate})
age_df

,나이,나이대
0,16,"10,20대"
1,4,10세이하
2,40,40대
3,8,10세이하
4,31,30대
5,16,"10,20대"
6,13,"10,20대"
7,6,10세이하
8,29,"10,20대"
9,31,30대


#### qcut
- N등분을 지정하면 동일한 원소의 개수로 N 개의 그룹으로 나눈다.

In [50]:
pd.qcut(ages, 3)

0     (14.0, 29.667]
1     (-0.001, 14.0]
2     (29.667, 47.0]
3     (-0.001, 14.0]
4     (29.667, 47.0]
5     (14.0, 29.667]
6     (-0.001, 14.0]
7     (-0.001, 14.0]
8     (14.0, 29.667]
9     (29.667, 47.0]
10    (-0.001, 14.0]
11    (-0.001, 14.0]
12    (-0.001, 14.0]
13    (29.667, 47.0]
14    (-0.001, 14.0]
15    (14.0, 29.667]
16    (29.667, 47.0]
17    (14.0, 29.667]
18    (29.667, 47.0]
19    (14.0, 29.667]
20    (29.667, 47.0]
21    (-0.001, 14.0]
22    (29.667, 47.0]
23    (29.667, 47.0]
24    (-0.001, 14.0]
25    (14.0, 29.667]
26    (14.0, 29.667]
27    (29.667, 47.0]
28    (14.0, 29.667]
29    (-0.001, 14.0]
dtype: category
Categories (3, interval[float64, right]): [(-0.001, 14.0] < (14.0, 29.667] < (29.667, 47.0]]

In [51]:
age_cate2 = pd.qcut(ages, 3, labels=['상', '중', '하'])
age_cate2

0     중
1     상
2     하
3     상
4     하
5     중
6     상
7     상
8     중
9     하
10    상
11    상
12    상
13    하
14    상
15    중
16    하
17    중
18    하
19    중
20    하
21    상
22    하
23    하
24    상
25    중
26    중
27    하
28    중
29    상
dtype: category
Categories (3, object): ['상' < '중' < '하']

In [52]:
df2 = pd.DataFrame({'나이':ages, '나이대':age_cate2})
df2

,나이,나이대
0,16,중
1,4,상
2,40,하
3,8,상
4,31,하
5,16,중
6,13,상
7,6,상
8,29,중
9,31,하


In [49]:
df2.groupby('나이대')['나이'].count()

나이대
상    11
중     9
하    10
Name: 나이, dtype: int64